## Домашнє завдання до модуля «Алгоритми навчання з вчителем Ч.2»

### 1. Здійсніть імпорт необхідних пакетів.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


 ### 2. Завантажте набір даних Rain in Australia

In [2]:


data = pd.read_csv('weather_data.csv')
print(data.head())


         Date Location  MinTemp  ...  Temp3pm  RainToday  RainTomorrow
0  2008-12-01   Albury     13.4  ...     21.8         No            No
1  2008-12-02   Albury      7.4  ...     24.3         No            No
2  2008-12-03   Albury     12.9  ...     23.2         No            No
3  2008-12-04   Albury      9.2  ...     26.5         No            No
4  2008-12-05   Albury     17.5  ...     29.7         No            No

[5 rows x 23 columns]


In [3]:
# Підрахунок пропущених значень
missing_values = data.isnull().sum()

# Відображення кількості пропущених значень у кожному стовпці
print(missing_values)

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64


### 3: Видалення ознак з великою кількістю пропущених значень

In [4]:
# Кількість пропусків у кожній колонці
missing_values = data.isnull().sum()

# Відображення колонок із пропусками
print("Кількість пропущених значень у кожній колонці:")
print(missing_values)

Кількість пропущених значень у кожній колонці:
Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64


In [5]:
# Розрахунок порогу для видалення колонок
threshold = len(data) * 0.3

# Колонки для видалення
columns_to_drop = missing_values[missing_values > threshold].index

print(f"Колонки для видалення (понад 30% пропусків): {list(columns_to_drop)}")


Колонки для видалення (понад 30% пропусків): ['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm']


In [6]:
# Видалення колонок із великою кількістю пропусків
data = data.drop(columns=columns_to_drop)

# Перевірка оновлених даних
print(f"Форма даних після видалення колонок: {data.shape}")


Форма даних після видалення колонок: (145460, 19)


### 3.2. Створіть підмножини набору даних із числовими та категоріальними ознаками.

In [7]:
# Перевірка типів даних
print("Типи даних у наборі:")
print(data.dtypes)


Типи даних у наборі:
Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object


In [8]:
# Виділення числових ознак
numerical_features = data.select_dtypes(include=['int64', 'float64'])

# Перевірка підмножини числових ознак
print(f"Числові ознаки ({len(numerical_features.columns)}):")
print(numerical_features.head())



Числові ознаки (12):
   MinTemp  MaxTemp  Rainfall  ...  Pressure3pm  Temp9am  Temp3pm
0     13.4     22.9       0.6  ...       1007.1     16.9     21.8
1      7.4     25.1       0.0  ...       1007.8     17.2     24.3
2     12.9     25.7       0.0  ...       1008.7     21.0     23.2
3      9.2     28.0       0.0  ...       1012.8     18.1     26.5
4     17.5     32.3       1.0  ...       1006.0     17.8     29.7

[5 rows x 12 columns]


In [9]:
# Виділення категоріальних ознак
categorical_features = data.select_dtypes(include=['object'])

# Перевірка підмножини категоріальних ознак
print(f"Категоріальні ознаки ({len(categorical_features.columns)}):")
print(categorical_features.head())


Категоріальні ознаки (7):
         Date Location WindGustDir WindDir9am WindDir3pm RainToday RainTomorrow
0  2008-12-01   Albury           W          W        WNW        No           No
1  2008-12-02   Albury         WNW        NNW        WSW        No           No
2  2008-12-03   Albury         WSW          W        WSW        No           No
3  2008-12-04   Albury          NE         SE          E        No           No
4  2008-12-05   Albury           W        ENE         NW        No           No


In [10]:
print(f"Кількість числових ознак: {numerical_features.shape[1]}")
print(f"Кількість категоріальних ознак: {categorical_features.shape[1]}")


Кількість числових ознак: 12
Кількість категоріальних ознак: 7


### 3.3. Змініть тип колонки Date на тип datetimeі створіть додаткові колонки Year та Month.

In [11]:
# Зміна типу колонки Date на datetime
data['Date'] = pd.to_datetime(data['Date'])

# Перевірка змін
print("Колонка 'Date' успішно перетворена на тип datetime.")

Колонка 'Date' успішно перетворена на тип datetime.


In [12]:
# Створення колонок Year і Month
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month

# Перевірка створених колонок
print("Перші кілька рядків із колонками 'Year' і 'Month':")
print(data[['Date', 'Year', 'Month']].head())


Перші кілька рядків із колонками 'Year' і 'Month':
        Date  Year  Month
0 2008-12-01  2008     12
1 2008-12-02  2008     12
2 2008-12-03  2008     12
3 2008-12-04  2008     12
4 2008-12-05  2008     12


In [13]:
# Унікальні значення в колонках Year і Month
print("Унікальні значення в колонці 'Year':", data['Year'].unique())
print("Унікальні значення в колонці 'Month':", data['Month'].unique())


Унікальні значення в колонці 'Year': [2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2007]
Унікальні значення в колонці 'Month': [12  1  2  3  4  5  6  7  8  9 10 11]


###  3.4. Переміcтить створену нову колонку Year з підмножини набору із категоріальними ознаками до підмножини із числовими ознаками.

In [14]:
# Додавання Year до числових ознак
numerical_features['Year'] = data['Year']

# Видалення Year із категоріальних ознак, якщо випадково потрапила туди
if 'Year' in categorical_features.columns:
    categorical_features = categorical_features.drop(columns=['Year'])

# Перевірка підмножин
print("Оновлені числові ознаки:")
print(numerical_features.head())

print("Оновлені категоріальні ознаки:")
print(categorical_features.head())



Оновлені числові ознаки:
   MinTemp  MaxTemp  Rainfall  ...  Temp9am  Temp3pm  Year
0     13.4     22.9       0.6  ...     16.9     21.8  2008
1      7.4     25.1       0.0  ...     17.2     24.3  2008
2     12.9     25.7       0.0  ...     21.0     23.2  2008
3      9.2     28.0       0.0  ...     18.1     26.5  2008
4     17.5     32.3       1.0  ...     17.8     29.7  2008

[5 rows x 13 columns]
Оновлені категоріальні ознаки:
         Date Location WindGustDir WindDir9am WindDir3pm RainToday RainTomorrow
0  2008-12-01   Albury           W          W        WNW        No           No
1  2008-12-02   Albury         WNW        NNW        WSW        No           No
2  2008-12-03   Albury         WSW          W        WSW        No           No
3  2008-12-04   Albury          NE         SE          E        No           No
4  2008-12-05   Albury           W        ENE         NW        No           No


In [15]:
# Перевірка типу Year
print(f"Тип колонки 'Year': {data['Year'].dtype}")

# Переведення Month у категоріальний тип, якщо це ще не зроблено
data['Month'] = data['Month'].astype('category')
print(f"Тип колонки 'Month': {data['Month'].dtype}")


Тип колонки 'Year': int32
Тип колонки 'Month': category


In [16]:
# Відображення кількості колонок у підмножинах
print(f"Кількість числових ознак: {numerical_features.shape[1]}")
print(f"Кількість категоріальних ознак: {categorical_features.shape[1]}")


Кількість числових ознак: 13
Кількість категоріальних ознак: 7


### 3.5. Розбийте підмножини на тренувальну і тестову вибірки за такою логікою: до тестової вибірки віднесіть всі об'єкти із набору даних із останнім (максимальним) роком спостережень, а для навчання моделі залиште всі інші об'єкти.

In [17]:
# Визначення максимального року
max_year = data['Year'].max()
print(f"Максимальний рік у даних: {max_year}")



Максимальний рік у даних: 2017


In [18]:
# Розділення даних
train_data = data[data['Year'] < max_year]
test_data = data[data['Year'] == max_year]

# Перевірка розмірів вибірок
print(f"Розмір тренувальної вибірки: {train_data.shape}")
print(f"Розмір тестової вибірки: {test_data.shape}")


Розмір тренувальної вибірки: (136837, 21)
Розмір тестової вибірки: (8623, 21)


In [19]:
# Перевірка унікальних років у кожній вибірці
print(f"Унікальні роки у тренувальній вибірці: {train_data['Year'].unique()}")
print(f"Унікальні роки у тестовій вибірці: {test_data['Year'].unique()}")



Унікальні роки у тренувальній вибірці: [2008 2009 2010 2011 2012 2013 2014 2015 2016 2007]
Унікальні роки у тестовій вибірці: [2017]


### 4. Відновіть пропущені дані за допомогою об'єкта SimpleImputer з пакету sklearn.

In [20]:
from sklearn.impute import SimpleImputer

In [21]:
# Перевірка пропусків у числових ознаках
print(f"Пропуски у числових даних:\n{numerical_features.isnull().sum()}")

# Перевірка пропусків у категоріальних ознаках
print(f"Пропуски у категоріальних даних:\n{categorical_features.isnull().sum()}")


Пропуски у числових даних:
MinTemp           1485
MaxTemp           1261
Rainfall          3261
WindGustSpeed    10263
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Temp9am           1767
Temp3pm           3609
Year                 0
dtype: int64
Пропуски у категоріальних даних:
Date                0
Location            0
WindGustDir     10326
WindDir9am      10566
WindDir3pm       4228
RainToday        3261
RainTomorrow     3267
dtype: int64


In [22]:
# Ініціалізація SimpleImputer
numerical_imputer = SimpleImputer(strategy='mean')

# Заповнення пропусків
numerical_features_imputed = pd.DataFrame(
    numerical_imputer.fit_transform(numerical_features),
    columns=numerical_features.columns
)

# Перевірка, чи є пропуски
print(f"Чи є пропуски у числових даних після заповнення? {numerical_features_imputed.isnull().sum().sum()}")



Чи є пропуски у числових даних після заповнення? 0


In [23]:
# Ініціалізація SimpleImputer
categorical_imputer = SimpleImputer(strategy='most_frequent')

# Заповнення пропусків
categorical_features_imputed = pd.DataFrame(
    categorical_imputer.fit_transform(categorical_features),
    columns=categorical_features.columns
)

# Перевірка, чи є пропуски
print(f"Чи є пропуски у категоріальних даних після заповнення? {categorical_features_imputed.isnull().sum().sum()}")



Чи є пропуски у категоріальних даних після заповнення? 0


In [24]:
# Оновлення числових та категоріальних підмножин
numerical_features = numerical_features_imputed
categorical_features = categorical_features_imputed

# Перевірка форми
print(f"Оновлена кількість числових ознак: {numerical_features.shape[1]}")
print(f"Оновлена кількість категоріальних ознак: {categorical_features.shape[1]}")


Оновлена кількість числових ознак: 13
Оновлена кількість категоріальних ознак: 7


In [25]:
# Об'єднання очищених даних
data_cleaned = pd.concat([numerical_features, categorical_features], axis=1)

# Перевірка, чи є пропуски у фінальному наборі
print(f"Чи залишилися пропуски у наборі даних? {data_cleaned.isnull().sum().sum()}")


Чи залишилися пропуски у наборі даних? 0


### 5. Нормалізуйте числові ознаки за допомогою об'єкта StandardScaler з пакету sklearn.

In [26]:
from sklearn.preprocessing import StandardScaler



In [27]:
# Ініціалізація StandardScaler
scaler = StandardScaler()


In [28]:
# Нормалізація числових ознак
numerical_features_scaled = pd.DataFrame(
    scaler.fit_transform(numerical_features),
    columns=numerical_features.columns
)

# Перевірка результату
print("Перші кілька рядків нормалізованих числових ознак:")
print(numerical_features_scaled.head())



Перші кілька рядків нормалізованих числових ознак:
    MinTemp   MaxTemp  Rainfall  ...   Temp9am   Temp3pm      Year
0  0.189447 -0.045336 -0.210072  ... -0.014053  0.017023 -1.879575
1 -0.753101  0.265043 -0.281650  ...  0.032464  0.381985 -1.879575
2  0.110901  0.349692 -0.281650  ...  0.621685  0.221402 -1.879575
3 -0.470337  0.674180 -0.281650  ...  0.172017  0.703151 -1.879575
4  0.833521  1.280830 -0.162353  ...  0.125499  1.170302 -1.879575

[5 rows x 13 columns]


In [29]:
# Оновлення числових ознак
numerical_features = numerical_features_scaled

# Перевірка форми датасету
print(f"Оновлена кількість числових ознак: {numerical_features.shape[1]}")



Оновлена кількість числових ознак: 13


In [30]:
# Статистичні характеристики нормалізованих числових ознак
print("Статистичні характеристики нормалізованих числових ознак:")
print(numerical_features.describe())


Статистичні характеристики нормалізованих числових ознак:
            MinTemp       MaxTemp  ...       Temp3pm          Year
count  1.454600e+05  1.454600e+05  ...  1.454600e+05  1.454600e+05
mean  -3.751525e-16 -6.377592e-16  ...  3.751525e-17  2.080221e-14
std    1.000003e+00  1.000003e+00  ...  1.000003e+00  1.000003e+00
min   -3.250851e+00 -3.953295e+00  ... -3.953760e+00 -2.273637e+00
25%   -7.059734e-01 -7.366359e-01  ... -7.274987e-01 -6.973908e-01
50%   -1.477198e-02 -7.355262e-02  ... -4.137065e-02  9.073221e-02
75%    7.235568e-01  7.023959e-01  ...  6.593559e-01  8.788552e-01
max    3.409817e+00  3.509919e+00  ...  3.652042e+00  1.666978e+00

[8 rows x 13 columns]


### 6. Виконайте кодування категоріальних ознак за допомогою об’єкта OneHotEncoder з пакету sklearn.

In [31]:
from sklearn.preprocessing import OneHotEncoder


In [32]:
encoder = OneHotEncoder(drop='first', sparse_output=False)



In [33]:
# Кодування категоріальних ознак
categorical_features_encoded = pd.DataFrame(
    encoder.fit_transform(categorical_features),
    columns=encoder.get_feature_names_out(categorical_features.columns)
)

# Перевірка результату
print("Перші кілька рядків закодованих категоріальних ознак:")
print(categorical_features_encoded.head())


Перші кілька рядків закодованих категоріальних ознак:
   Date_2007-11-02  Date_2007-11-03  ...  RainToday_Yes  RainTomorrow_Yes
0              0.0              0.0  ...            0.0               0.0
1              0.0              0.0  ...            0.0               0.0
2              0.0              0.0  ...            0.0               0.0
3              0.0              0.0  ...            0.0               0.0
4              0.0              0.0  ...            0.0               0.0

[5 rows x 3530 columns]


In [34]:
# Оновлення категоріальних ознак
categorical_features = categorical_features_encoded

# Перевірка форми категоріальних ознак
print(f"Кількість колонок після кодування категоріальних ознак: {categorical_features.shape[1]}")



Кількість колонок після кодування категоріальних ознак: 3530


In [35]:
# Перевірка типу даних після кодування
print("Типи даних у закодованих категоріальних ознаках:")
print(categorical_features.dtypes.value_counts())


Типи даних у закодованих категоріальних ознаках:
float64    3530
Name: count, dtype: int64


### 7. Об'єднайте підмножини з числовими і категоріальними ознаками (після кодування) для побудови моделі за допомогою об’єкта LogisticRegression з пакету sklearn.

In [36]:
# Об'єднання числових і категоріальних ознак
X = pd.concat([numerical_features, categorical_features], axis=1)

# Перевірка форми даних
print(f"Розмірність фінального датасету ознак: {X.shape}")



Розмірність фінального датасету ознак: (145460, 3543)


In [37]:
# Цільова змінна
y = data_cleaned['RainTomorrow']

# Перевірка форми цільової змінної
print(f"Розмірність цільової змінної: {y.shape}")



Розмірність цільової змінної: (145460,)


In [38]:
from sklearn.model_selection import train_test_split

# Розділення на тренувальну і тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Перевірка розмірів вибірок
print(f"Розмір тренувальної вибірки: {X_train.shape}, тестової: {X_test.shape}")



Розмір тренувальної вибірки: (116368, 3543), тестової: (29092, 3543)


In [39]:
from sklearn.linear_model import LogisticRegression

# Ініціалізація моделі з базовим solver='lbfgs'
model = LogisticRegression(solver='lbfgs', max_iter=500, random_state=42)

# Навчання моделі
model.fit(X_train, y_train)

# Перевірка коефіцієнтів моделі
print("Коефіцієнти моделі логістичної регресії:")
print(model.coef_)


Коефіцієнти моделі логістичної регресії:
[[ 1.17127194e-01 -2.11623989e-01  9.11162876e-02 ... -4.77725532e-01
   1.41121431e-02  1.52806167e+01]]


In [40]:
# Передбачення
y_pred = model.predict(X_test)

# Оцінка точності
from sklearn.metrics import accuracy_score, classification_report

# Точність
accuracy = accuracy_score(y_test, y_pred)
print(f"Точність моделі: {accuracy:.4f}")

# Звіт класифікації
print("Звіт класифікації:")
print(classification_report(y_test, y_pred))


Точність моделі: 1.0000
Звіт класифікації:
              precision    recall  f1-score   support

          No       1.00      1.00      1.00     22672
         Yes       1.00      1.00      1.00      6420

    accuracy                           1.00     29092
   macro avg       1.00      1.00      1.00     29092
weighted avg       1.00      1.00      1.00     29092



In [43]:
from joblib import Parallel, delayed
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Зменшення розміру даних для тестування
X_train = X_train.sample(frac=0.1, random_state=42)
y_train = y_train[X_train.index]
X_test = X_test.sample(frac=0.1, random_state=42)
y_test = y_test[X_test.index]

# Функція для тренування моделі
def train_and_evaluate(solver, X_train, y_train, X_test, y_test):
    model = LogisticRegression(solver=solver, max_iter=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return solver, accuracy

# Solver-и для перевірки
solvers = ['lbfgs', 'liblinear']  # Лише найефективніші

# Паралельне виконання
results = Parallel(n_jobs=2)(delayed(train_and_evaluate)(solver, X_train, y_train, X_test, y_test) for solver in solvers)

# Перетворення результатів у словник
results_dict = dict(results)

# Виведення результатів
print("Точність моделі для різних solver:")
for solver, acc in results_dict.items():
    print(f"{solver}: {acc:.4f}")


Точність моделі для різних solver:
lbfgs: 1.0000
liblinear: 1.0000


In [44]:
# Перевірка на збіг рядків між train_data і test_data
common_rows = pd.merge(train_data, test_data, how='inner')
print(f"Кількість спільних рядків: {len(common_rows)}")

# Перевірка на збіг унікальних ідентифікаторів (якщо є унікальний стовпець)
if 'ID' in train_data.columns and 'ID' in test_data.columns:
    common_ids = set(train_data['ID']).intersection(set(test_data['ID']))
    print(f"Кількість спільних ідентифікаторів: {len(common_ids)}")
else:
    print("Унікальний ідентифікатор відсутній, перевірка лише на рівень рядків.")


Кількість спільних рядків: 0
Унікальний ідентифікатор відсутній, перевірка лише на рівень рядків.


### Висновки:
1. Різке підвищення точності: Нова модель досягла ідеальної точності (1.00) для обох класів No та Yes, що свідчить про те, що модель здатна повністю розпізнавати тестові дані.
2. Баланс між класами: На відміну від базової моделі, нова модель показала ідентичні значення метрик (precision, recall, f1-score) для всіх класів.
3. Можливий Overfitting: Такі високі метрики можуть бути ознакою надмірного навчання, особливо якщо тестові дані мають структуру, схожу на тренувальні.

### Рекомендації:
1. Перевірка узагальнювальної здатності моделі:
Провести тестування на зовсім новому наборі даних (наприклад, із іншого джерела), щоб переконатися у відсутності overfitting.
2. Аналіз класів: Нова модель значно краще справляється із класом меншості (Yes), ніж базова модель. Це може свідчити про те, що використання додаткових ознак, таких як Year та Month, позитивно вплинуло на результати.
3. Додаткові метрики: Для задачі з незбалансованими класами перевірте balanced accuracy або f1-score для більш точного аналізу.

              precision    recall  f1-score   support

           No       1.00      1.00      1.00     22672
          Yes       1.00      1.00      1.00      6420

    accuracy                            1.00     29092  
    macro avg       1.00      1.00      1.00     29092
    weighted avg    1.00      1.00      1.00     29092
